In [1]:
import os
import json
import numpy as np
from PIL import Image, ImageEnhance

# Paths to directories
specific_dates = ['20240718']         # use variable below when you have all geoJSON 
# specific_dates = ['20240718', '20230927', '20231108', '20240314', '20240516', '20240626']
base_tif_dir = './data/campaigns'
geojson_dir = './geojson_file'

output_image_dir_with_contrast = 'dataset/full_images/images_with_contrast'
output_image_dir_no_contrast = 'dataset/full_images/images_no_contrast'
output_label_dir = 'dataset/labels_full_images'

# Create output directories
os.makedirs(output_image_dir_with_contrast, exist_ok=True)
os.makedirs(output_image_dir_no_contrast, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Image dimensions for YOLO normalization
IMAGE_WIDTH = 1616
IMAGE_HEIGHT = 1240

def get_geojson_files(geojson_dir):
    """
    Get set of geoJSON file names without extensions.
    """
    return {os.path.splitext(f)[0] for f in os.listdir(geojson_dir) if f.endswith('.geojson')}

def create_yolo_annotation(file_path, class_id, x_center, y_center, width, height):
    """
    Append YOLO format annotation to a text file.
    """
    with open(file_path, 'a') as file:  # Open in append mode to add multiple objects
        file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

def process_geojson_file(file_name, tif_dir):
    """
    Process a single geoJSON file and its corresponding .tif file.
    """
    tif_path = os.path.join(tif_dir, file_name + '.tif')
    geojson_path = os.path.join(geojson_dir, file_name + '.geojson')

    # Check if the corresponding .tif file exists
    if not os.path.exists(tif_path):
        print(f"Warning: `{tif_path}` not found. Skipping this file.")
        return

    # Load geoJSON data
    with open(geojson_path, 'r') as file:
        geojson_data = json.load(file)

    # Open the .tif file and process each frame specified in the geoJSON
    with Image.open(tif_path) as tif_img:
        for feature in geojson_data['features']:
            process_feature(feature, tif_img, file_name)

def process_feature(feature, tif_img, file_name):
    """
    Process an individual feature within a geoJSON file.
    """
    class_mapping = {"Living": 0, "Non-Living": 1, "Bubble": 2}
    classification = feature['properties']['classification']['name']
    class_id = class_mapping.get(classification, class_mapping["Non-Living"])
    coordinates = feature['geometry']['coordinates'][0]
    frame_idx = feature['geometry']['plane']['t']

    # Access the specified frame in the .tif stack
    tif_img.seek(frame_idx)
    frame = np.array(tif_img)

    # Define filenames for both contrast and no-contrast images
    image_filename = f"{file_name}_frame_{frame_idx}.png"
    image_path_with_contrast = os.path.join(output_image_dir_with_contrast, image_filename)
    image_path_no_contrast = os.path.join(output_image_dir_no_contrast, image_filename)

    # Save the no-contrast full image if it hasn't been saved yet
    if not os.path.exists(image_path_no_contrast):
        Image.fromarray(frame).save(image_path_no_contrast)

    # Save the high-contrast image if it hasn't been saved yet
    if not os.path.exists(image_path_with_contrast):
        # Convert to PIL image and adjust contrast
        pil_frame = Image.fromarray(frame)
        enhancer = ImageEnhance.Contrast(pil_frame)
        enhanced_image = enhancer.enhance(30)  # Increase contrast by a high factor
        enhanced_image.save(image_path_with_contrast)

    # Calculate YOLO bounding box coordinates
    x_coords = [point[0] for point in coordinates]
    y_coords = [point[1] for point in coordinates]
    min_x, max_x = int(min(x_coords)), int(max(x_coords))
    min_y, max_y = int(min(y_coords)), int(max(y_coords))
    
    x_center = (min_x + max_x) / 2 / IMAGE_WIDTH
    y_center = (min_y + max_y) / 2 / IMAGE_HEIGHT
    width = (max_x - min_x) / IMAGE_WIDTH
    height = (max_y - min_y) / IMAGE_HEIGHT
    
    # Write annotation to a .txt file for the full image, appending each object in the frame
    annotation_filename = os.path.splitext(image_filename)[0] + ".txt"
    annotation_path = os.path.join(output_label_dir, annotation_filename)
    create_yolo_annotation(annotation_path, class_id, x_center, y_center, width, height)

def main():
    """
    Main function to process geoJSON and TIFF files across all specified dates.
    """
    for date in specific_dates:
        tif_dir = os.path.join(base_tif_dir, date, 'image_stacks')
        print(f"Processing files for date: {date}")
        geojson_files = get_geojson_files(geojson_dir)
        for file_name in geojson_files:
            process_geojson_file(file_name, tif_dir)

if __name__ == "__main__":
    main()

Processing files for date: 20240718
